<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D4_2_Pronostico.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Predicción del clima

Este notebook introduce la predicción en series temporales. Las aplicaciones de este tipo de predicción son muy diversas, como detección de arritmias cardíacas hasta predecir fluctuaciones de precios en la bolsa.

Primero que nada, importamos las bibliotecas necesarias. En esta ocasión, preste mucha atención, pues usaremos más bibliotecas que en ejemplos anteriores.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
# importamos Pandas, que nos permitirá ver mejor los datos
import pandas as pd

# configuramos las figuras de matplotlib
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
plt.style.use('default')

# Importar el dataset

El [*dataset*](https://www.bgc-jena.mpg.de/wetter/) que usamos en este caso es del clima y fue recopilado por el [Max Planck Institute for Biogeochemistry](https://www.bgc-jena.mpg.de/index.php/Main/HomePage). Contiene 14 variables diferentes como temperatura, presión atmosférica, humedad, etc. 

Fueron recolectados cada 10 minutos desde el 2003. Nosotros usaremos datos desde el 2009 hasta el 2016 por eficiencia. 

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

con el dataset ya en el disco duro, podemos importarlo. Para ello, usamos Pandas, que parsea y convenientemente nos deja explorar el dataset. 

In [ ]:
df = pd.read_csv(csv_path)
df.head()

Como podemos observar, tenemos 14 variables numéricas, que van cambiando con el tiempo. Esto nos permitirá modelar la evolución del clima. 

Veamos el tamaño de nuestro dataset. Podemos además usar funcionalidades de pandas para tener una idea de la distribución de cada una de las variables

In [ ]:
df.describe()

aqui podemos ver con claridad los diferentes rangos y distribuciones de cada una de las variables. Es siempre importante familiarizarse con los datos antes de saltar a probar modelos de predicción. Algunas preguntas útiles en este punto son:

* ¿Tiene sentido el rango de esta variable?
* ¿Es este el mejor tipo de dato para representar este valor?

# Preparar un *dataset* univariable

Un aspecto interesante de las series temporales es la multitud de formas en que podemos desear predecir. En cada caso, los ejemplos de entrenamiento tienen que ser cuidadosamente preparados para maximizar el éxito de la tarea de predicción. 

Supongamos que nuestra tarea es predecir la temperatura de las siguientes 6 horas. Para ello, elegimos considerar los datos de los 5 días previos. 

Veamos como podemos construir un dataset para esta tarea.

In [ ]:
def construir_dataset_univariable(dataset, inicio, fin, ancho_ventana, offset_prediccion):
    # definimos las variables donde se guardará el dataset
    data = []
    labels = []

    # el inicio y el fin determinan los límites en el tiempo que vamos a analizar
    # como vamos a ir mirando "hacia atras" sumamos el tamaño de la ventana previa
    # al primer punto temporal, de tal manera a movernos correctamente al 
    # extraer los ejemplos y labels
    inicio = inicio + ancho_ventana 
    if fin is None:
        fin = len(dataset) - offset_prediccion

    # recorremos la serie temporal para ir extrayendo ejemplos
    for i in range(inicio, fin):
        # definimos los indices de la ventana previa (datos que usaremos para predecir el futuro inmediato)
        ventana = range(i-ancho_ventana, i)

        # recolectamos la ventana previa a lo que queremos predecir
        # nos aseguramos que nuestro dato tenga la dimensionalidad correcta (univariable)
        data.append(np.reshape(dataset[ventana], (ancho_ventana, 1)))

        # recolectamos la "respuesta" (la ventana de datos que queremos predecir)
        labels.append(dataset[i+offset_prediccion])
    
    return np.array(data), np.array(labels)

En estas predicciones, usaremos los primeros 300000 ejemplos como *training set*, y el resto como *testing set*. Al usar series temporales se debe prestar mucha atención al hacer la partición del dataset, si queremos predecir el futuro, no es bueno "conocer el futuro" (usar los datos más recientes para el *training*).

In [ ]:
particion = 300000

# Predicción de temperatura

con nuestra función para construir ejemplos a partir del dataset, podemos empezar a hacer un pronóstico de temperatura. Primeramente, vamos a aislar los datos que nos interesan en este caso.

In [ ]:
# extraemos los datos de temperatura (en grados centígrados)
temp_data = df['T (degC)']
# hacemos que el índice de nuestro DataFrame de pandas sea la fecha del registro de temperatura
temp_data.index = df['Date Time']
# visualizamos nuestro dataset filtrado
temp_data.head()

y observemos como se ve la serie temporal

In [ ]:
temp_data.plot(subplots=True)

Como queremos lidiar solamente con un array de numpy y no un DataFrame de pandas, extraemos todos los valores, pues ya estan correctamente ordenados.

In [ ]:
temp_data = temp_data.values

como siempre, es conveniente que normalicemos nuestro *dataset*. Hay muchas formas de normalizar. Nosotros vamos a usar el [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) de la biblioteca scikit-learn. Esta normalización remueve la media y divide entre la desviación estándar.

En este caso, es importante que nuestro *dataset* tenga dimensionalidad `(n_ejemplos, n_variables)`, pues es requerido por el `StandardScaler`. 

In [ ]:
print(f'dimensionalidad inicial: {temp_data.shape}')
temp_data = temp_data.reshape(temp_data.shape[0], 1)
print(f'dimensionalidad final: {temp_data.shape}')


**Note:** Esto es un comportamiento específico de numpy, y un requerimiento específico del `StandardScaler`.

Es importante notar que el escalador solo debe ajustarse al *training set*, pero el *testing set* debe ser normalizado igualmente. Es decir, A ambos sets se resta el promedio y se divide la desviación estándar del *training set*.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# por ahora solo extraemos el training set, que se usa para calcular los parámetros para normalizar
training_data = temp_data[:particion] 
scaler.fit(training_data)

# normalizamos todo el dataset
temp_data = scaler.transform(temp_data)

exploremos de nuevo nuestro dataset

In [ ]:
temp_data

In [ ]:
plt.plot(temp_data)

podemos ver que la figura cambio muy poco, en particular, estamos en un rango mucho más chico. También se puede hacer una normalización entre 0 y 1 usando el `MinMaxScaler`.

Creemos ahora los ejemplos de *training* y *testing* usando la función que definimos más arriba.

In [ ]:
ancho_ventana = 20
prediccion = 0

# extraemos los ejemplos de training
x_train, y_train = construir_dataset_univariable(temp_data, 0, particion, ancho_ventana, prediccion)

# extraemos los ejemplos de testing
x_test, y_test = construir_dataset_univariable(temp_data, particion, None, ancho_ventana, prediccion)

exploremos un ejemplo de nuestro *training set*

In [ ]:
print('historia de temperatura')
print(x_train[0])
print('temperatura a predecir')
print(y_train[0])

podemos visualizar mejor el ejemplo si lo dibujamos como serie temporal

In [ ]:
def ver_ejemplo(ejemplo, delta, titulos):
    rotulos = ['Historia', 'Futuro verdadero'] + titulos[1:]
    marker = ['.-', 'rx', 'go', 'mD', 'c<']
    tiempos = list(range(-ejemplo[0].shape[0], 0))
    if delta:
        futuro = delta
    else:
        futuro = 0
    
    plt.title(titulos[0])
    for i, x in enumerate(ejemplo):
        if i:  # futuro verdadero o predicción
            plt.plot(futuro, ejemplo[i], marker[i], markersize=10, label=rotulos[i])
        else:  # historia
            plt.plot(tiempos, ejemplo[i], marker[i], label=rotulos[i])
    plt.legend()
    plt.xlim([tiempos[0], (futuro+5)*2])
    plt.xlabel('Tiempo')
    return plt

vemos el primer ejemplo

In [ ]:
ver_ejemplo([x_train[0], y_train[0]], 0, ['Ejemplo 0'])

# Modelo simple (baseline)

Para poder comparar nuestro modelo con un modelo simple que puede servirnos de base, definimos una función que predice el futuro haciendo un promedio de los valores históricos.

In [ ]:
def modelo_promedio(historia):
    return np.mean(historia)

veamos que tan bien podemos predecir usando este simple modelo

In [ ]:
ver_ejemplo([x_train[0], y_train[0], modelo_promedio(x_train[0])], 0, ['Ejemplo 0', 'Promedio histórico'])

# Red neuronal recurrente

Vamos a ver si podemos superar al valor histórico promedio usando una red neuronal recurrente.

En este caso, introducimos un tipo de *layer* recurrente: Long Short Term Memory ([LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM))

Primero que nada, veamos como podemos preparar un dataset de tensorflow que separe nuestros ejemplos en batches. Hay 3 cosas importantes que introducimos en este caso:

* **cache:** optimiza el acceso de tensorflow a los ejemplos en memoria
* **shuffle:** randomiza el orden en que los ejemplos se incluyen en el dataset.
* **batch:** determina cuantos ejemplos del dataset se usan antes de actualizar los pesos de la red neuronal.


In [ ]:
batch = 256
buffer = 10000

# creamos el training dataset
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train = train.cache().shuffle(buffer).batch(batch).repeat()

# creamos el testing dataset
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test = test.batch(batch).repeat()

Si desea entender como funcionan los comandos `batch` y `repeat` le recomiendo leer [esta respuesta en stackoverflow](https://stackoverflow.com/a/53517848/943138) (en inglés).

# Crear la red neuronal

vamos a crear una red neuronal recurrente relativamente sencilla. 

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.LSTM(8, input_shape=x_train.shape[-2:]),
                                    tf.keras.layers.Dense(1)])

model.compile(optimizer='adam', loss='mae')

entrenar la red neuronal.

Introducimos nuevos parámetros al momento de entrenar.
* `stops_per_epoch`: va a correr por 200 pasos (batches), y no por todo el dataset en cada *epoch*
* `validation_steps`: la cantidad de batches del *testing set* que se usan para medir la función de costo de evaluación en cada *epoch*

In [ ]:
model.fit(train, epochs=10, steps_per_epoch=200, validation_data=test, validation_steps=50)

veamos algunas predicciones comparando con el modelo que predice el promedio histórico.

In [ ]:
x.numpy().shape

In [ ]:
for x,y in test.take(4):
    ver_ejemplo([x[0].numpy() ,y[0].numpy(), model.predict(x)[0], modelo_promedio(x)], 0, 
                ['Comparación de modelos', 'shallow LSTM', 'Promedio histórico'])
    plt.show()

# Créditos

Este notebook traduce y adapta el código y explicaciones del [Tutorial de Tensorflow](https://www.tensorflow.org/tutorials/structured_data/time_series) en series temporales.